In [ ]:
!pip install datasets pandas
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

# Predict editing effort (edit time or HTER) based on different features

## Load dataset

In [2]:
data = load_dataset("GroNLP/divemt")
print(data)
data["train"].to_pandas().head()
data_p = data["train"].to_pandas()
data_p_pe = data_p.loc[data_p['task_type'] != "ht"].reset_index()

Generating train split: 0 examples [00:00, ? examples/s]

Dataset divemt downloaded and prepared to /root/.cache/huggingface/datasets/GroNLP___divemt/main/1.0.0/c875a08adbd66d5d9c6d8c79f7816092829b41f855641161190c69e19b4f11b1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['unit_id', 'flores_id', 'item_id', 'subject_id', 'lang_id', 'doc_id', 'task_type', 'translation_type', 'src_len_chr', 'mt_len_chr', 'tgt_len_chr', 'src_len_wrd', 'mt_len_wrd', 'tgt_len_wrd', 'edit_time', 'k_total', 'k_letter', 'k_digit', 'k_white', 'k_symbol', 'k_nav', 'k_erase', 'k_copy', 'k_cut', 'k_paste', 'k_do', 'n_pause_geq_300', 'len_pause_geq_300', 'n_pause_geq_1000', 'len_pause_geq_1000', 'event_time', 'num_annotations', 'last_modification_time', 'n_insert', 'n_delete', 'n_substitute', 'n_shift', 'tot_shifted_words', 'tot_edits', 'hter', 'cer', 'bleu', 'chrf', 'time_s', 'time_m', 'time_h', 'time_per_char', 'time_per_word', 'key_per_char', 'words_per_hour', 'words_per_minute', 'per_subject_visit_order', 'src_text', 'mt_text', 'tgt_text', 'aligned_edit', 'src_tokens', 'src_annotations', 'mt_tokens', 'mt_annotations', 'tgt_tokens', 'tgt_annotations', 'src_wmt22_qe', 'mt_wmt22_qe'],
        num_rows: 7740
    })
})


## Functions to compute features

In [3]:
# HEAD related features
def get_mean_head_dist(data, level="src"):
    heads = data[f"{level}_annotations"]["head"]
    head_dists = np.asarray([abs(index - int(head)) for index, head in enumerate(heads)])
    return np.sum(head_dists)/len(heads)

def get_median_head_dist(data, level="src"):
    heads = data[f"{level}_annotations"]["head"]
    head_dists = np.asarray([abs(index - int(head)) for index, head in enumerate(heads)])
    return np.median(head_dists)


# POS related features
def get_pos_rate(data, more_pos=["NOUN"]):
    all_pos_src = data["src_annotations"]["upos"]
    all_pos_mt = data["mt_annotations"]["upos"]
    all_pos_tgt = data["tgt_annotations"]["upos"]
    
    this_pos_src = [np.count_nonzero(all_pos_src == pos) for pos in more_pos]
    this_pos_mt = [np.count_nonzero(all_pos_mt == pos) for pos in more_pos]
    this_pos_tgt = [np.count_nonzero(all_pos_tgt == pos) for pos in more_pos]

    ratios_src = [this_count / len(all_pos_src) for this_count in this_pos_src]
    ratios_mt = [this_count / len(all_pos_mt) for this_count in this_pos_mt]
    ratios_tgt = [this_count / len(all_pos_tgt) for this_count in this_pos_tgt]

    ratio1 = [r_mt / r_src if r_src > 0 else 0 for r_mt, r_src in zip(ratios_mt, ratios_src)]
    ratio2 = [r_mt / r_tgt if r_tgt > 0 else 0 for r_mt, r_tgt in zip(ratios_mt, ratios_tgt)]

    ratio = [val for pair in zip(ratio1, ratio2) for val in pair]
    return ratio


# features related to BAD-OK labels
def get_bad_rate(data_p_pe, level="src"):
    return data_p_pe.apply(lambda x: np.count_nonzero(x[f"{level}_wmt22_qe"] == "BAD") / len(x[f"{level}_wmt22_qe"]), axis=1)

def get_bad_ratio(data_p_pe):
    return data_p_pe.apply(lambda x: np.count_nonzero(x["mt_wmt22_qe"] == "BAD") / np.count_nonzero(x["src_wmt22_qe"] == "BAD") if np.count_nonzero(x["src_wmt22_qe"] == "BAD") != 0 else 1, axis=1)


#features related to tokens
def get_diff_tokens(data_p_pe, levels=["mt", "tgt"]):
    return (len(data_p_pe[f"{levels[0]}_tokens"]) - len(data_p_pe[f"{levels[1]}_tokens"])) / data_p_pe[f"{levels[1]}_len_wrd"]

def get_tokens_ratio(data_p_pe, levels=["mt", "tgt"]):
    return len(data_p_pe[f"{levels[0]}_tokens"]) / len(data_p_pe[f"{levels[1]}_tokens"]) if len(data_p_pe[f"{levels[1]}_tokens"]) != 0 else 0


## Add features to dataset

In [4]:
data_p_pe["src_bad_rate"] =  get_bad_rate(data_p_pe, level="src")
data_p_pe["mt_bad_rate"] =  get_bad_rate(data_p_pe, level="mt")
data_p_pe["bad_rate_ratio"] = get_bad_ratio(data_p_pe)

data_p_pe["median_head_distance"] = data_p_pe.apply(get_median_head_dist, axis=1)
data_p_pe["mean_head_distance"] = data_p_pe.apply(get_mean_head_dist, axis=1)

languages = np.unique(data_p_pe["lang_id"])
data_p_pe["lang_id_enc"] = data_p_pe.apply(lambda x: np.where(languages == x["lang_id"])[0][0], axis=1)

# data_p_pe["diff_tokens_tgt"] = get_diff_tokens(data_p_pe, levels=["mt", "tgt"])

data_p_pe["ratio_tokens_tgt"] = get_tokens_ratio(data_p_pe, levels=["mt", "tgt"])
data_p_pe["diff_tokens_src"] = get_diff_tokens(data_p_pe, levels=["mt", "src"])
data_p_pe["ratio_tokens_src"] = get_tokens_ratio(data_p_pe, levels=["mt", "src"])

pos_features = ["NOUN"]
pos_features_names = [f'{feature}_{i}' for feature in pos_features for i in range(1, 3)]
data_p_pe[pos_features_names] = data_p_pe.apply(lambda data: get_pos_rate(data, pos_features), axis=1, result_type="expand")


## Select which features to use

In [5]:
response = "hter" # time_s or hter

# source complexity and target fluency features
features_baseline = ["src_len_wrd", "mt_len_wrd", "tgt_len_wrd"]

# adding adequacy features
features_1 = ["src_len_wrd", "mt_len_wrd", "tgt_len_wrd", 
            "src_bad_rate", "mt_bad_rate", "bad_rate_ratio",
            "diff_tokens_src", "ratio_tokens_src", "ratio_tokens_tgt"]

# adding dequacy features
features_2 = ["src_len_wrd", "mt_len_wrd", "tgt_len_wrd", 
            "src_bad_rate", "mt_bad_rate", "bad_rate_ratio",
            "diff_tokens_src", "ratio_tokens_src", "ratio_tokens_tgt"] + pos_features_names

In [6]:
# get the correlation coefficient and p-value for time and hter
corr_coef, p_value = pearsonr(data_p_pe['time_s'], data_p_pe['hter'])

print("Correlation coefficient and p-value between time and hter:")
print(corr_coef, p_value)

Correlation coefficient and p-value between time and hter:
0.12833260405435876 2.150545006559888e-20


## Train model

In [7]:
train_df, test_df = train_test_split(data_p_pe, test_size=0.2, random_state=42)

response_train = train_df[response]
response_test = test_df[response]

# baseline
predictors_train_baseline = train_df[features_baseline]
predictors_test_baseline = test_df[features_baseline]

lin_reg_model_baseline = linear_model.LinearRegression()

reg_basline = lin_reg_model_baseline.fit(predictors_train_baseline, response_train)
pred_baseline = lin_reg_model_baseline.predict(predictors_test_baseline)

mse_basline = mean_squared_error(response_test, pred_baseline, squared=False)
r2_baseline = r2_score(response_test, pred_baseline)

print("Baseline")
print("Mean squared error: {:.5f}".format(mse_basline))
print("R-squared: {:.5f}".format(r2_baseline))


# features 1
predictors_train_1 = train_df[features_1]
predictors_test_1 = test_df[features_1]

lin_reg_model_1 = linear_model.LinearRegression()

reg_1 = lin_reg_model_1.fit(predictors_train_1, response_train)
pred_1 = lin_reg_model_1.predict(predictors_test_1)

mse_1 = mean_squared_error(response_test, pred_1, squared=False)
r2_1 = r2_score(response_test, pred_1)

print("features 1")
print("Mean squared error: {:.5f}".format(mse_1))
print("R-squared: {:.5f}".format(r2_1))


# features 2
predictors_train_2 = train_df[features_2]
predictors_test_2 = test_df[features_2]

lin_reg_model_2 = linear_model.LinearRegression()

reg_2 = lin_reg_model_2.fit(predictors_train_2, response_train)
pred_2 = lin_reg_model_2.predict(predictors_test_2)

mse_2 = mean_squared_error(response_test, pred_2, squared=False)
r2_2 = r2_score(response_test, pred_2)

print("features 2")
print("Mean squared error: {:.5f}".format(mse_2))
print("R-squared: {:.5f}".format(r2_2))


# OLS regression with more info
X2 = sm.add_constant(predictors_train_2)
est = sm.OLS(response_train, X2)
est2 = est.fit()
print(est2.summary())

Baseline
Mean squared error: 24.22339
R-squared: 0.03917
features 1
Mean squared error: 7.69589
R-squared: 0.90302
features 2
Mean squared error: 7.69117
R-squared: 0.90314
                            OLS Regression Results                            
Dep. Variable:                   hter   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     4410.
Date:                Sun, 02 Apr 2023   Prob (F-statistic):               0.00
Time:                        16:21:39   Log-Likelihood:                -14423.
No. Observations:                4128   AIC:                         2.886e+04
Df Residuals:                    4119   BIC:                         2.892e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std er